In [24]:
import os

topic_file = os.getenv("TOPIC_PATH")
users_file = os.getenv("USERS_PATH")
ratings_file = os.getenv("RATINGS_PATH")

#topic_file = "/data/books_data/books.csv"
#users_file = "/data/books_data/users.csv"
#ratings_file = "/data/books_data/rating.csv"

In [25]:
import pandas as pd
topics = pd.read_csv(topic_file)

In [26]:
topics["ID"] = pd.to_numeric(topics["ID"], errors='coerce')

In [27]:
topics.head()

,ID,NAME,CATEGORY
0,1,Decision in Normandy,Carlo D'Este
1,3,What If?: The World's Foremost Military Histor...,Robert Cowley
2,4,Beloved (Plume Contemporary Fiction),Toni Morrison
3,5,Our Dumb Century: The Onion Presents 100 Years...,The Onion
4,7,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish


In [28]:
ratings = pd.read_csv(ratings_file)
ratings.head()

,UserID,MovieID,Rating
0,1,6264,7.0
1,1,4350,7.0
2,1,6252,5.0
3,1,202,9.0
4,1,6266,6.0


In [29]:
users = pd.read_csv(users_file)
users.head()

,Gender,Age,UserID
0,"minneapolis, minnesota, usa",24,1
1,"san diego, california, usa",20,2
2,"novinger, missouri, usa",16,3
3,"sonoma, california, usa",34,4
4,"berkeley, california, usa",23,5


In [30]:
user_rating = pd.merge(users,ratings,on='UserID')
user_rating_mv = pd.merge(user_rating, topics, left_on="MovieID", right_on="ID") 

In [31]:
user_rating_mv

,Gender,Age,UserID,MovieID,Rating,ID,NAME,CATEGORY
0,"minneapolis, minnesota, usa",24,1,6264,7.0,6264,Something Wicked This Way Comes,Ray Bradbury
1,"west fairlee, vermont, usa",43,496,6264,8.0,6264,Something Wicked This Way Comes,Ray Bradbury
2,"mililani, hawaii, usa",46,676,6264,10.0,6264,Something Wicked This Way Comes,Ray Bradbury
3,"hamilton, ohio, usa",36,1842,6264,10.0,6264,Something Wicked This Way Comes,Ray Bradbury
4,"minneapolis, minnesota, usa",24,1,4350,7.0,4350,The Mists of Avalon,MARION ZIMMER BRADLEY
...,...,...,...,...,...,...,...,...
59455,"lake george, new york, usa",34,2945,16981,7.0,16981,"Jessi Ramsey, Pet Sitter (Baby-Sitters Club (P...",Ann M. Martin
59456,"lake george, new york, usa",34,2945,14501,7.0,14501,STAY AWAY FROM THE TREEHOUSE (GHOSTS OF FEAR S...,R.L. Stine
59457,"lake george, new york, usa",34,2945,3490,8.0,3490,Bright Eyes (Coulter Family Series),Catherine Anderson
59458,"lake george, new york, usa",34,2945,15719,8.0,15719,Arcadia,Jim Crace


In [32]:
#df = pd.DataFrame().reset_index()

top_movies = user_rating_mv[['UserID','Gender','Age', 'NAME']].groupby(['Gender','Age','NAME']).count().reset_index()

In [33]:
df1 = top_movies.query('Gender=="M" & Age=="25"').sort_values('UserID', ascending=False)
df1['NAME'].head(10)

Series([], Name: NAME, dtype: object)

In [34]:
def fetch_top_movies(age, gender):
    global top_movies
    qry = 'Gender=="'+gender+'" & Age=="'+age+'"'
    df1 = top_movies.query(qry).sort_values('UserID', ascending=False)
    return df1.head(10).NAME.to_list()

In [37]:
print(fetch_top_movies('24','minneapolis, minnesota, usa'))

['A Coral Kiss', "An Arrow's Flight", 'What a Wonderful World: A Lifetime of Recordings', 'To Marry McAllister  (Bachelor Cousins) (Harlequin Presents, No. 2273)', 'The Subtle Knife (His Dark Materials, Book 2)', 'The Mists of Avalon', 'The 10th Kingdom (Hallmark Entertainment Books)', 'Something Wicked This Way Comes', 'Silverlight: The Arcana, Book II (Arcana/Morgan Llywelyn, Bk 2)', 'Sideways Stories from Wayside School']


In [12]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours
import numpy
import pandas as pd

@scoring_func
def score(model, request):
    payload = request.json["payload"]
    print(payload)     
    global top_movies
    qry = 'Gender=="'+payload["gender"]+'" & Age=="'+payload["age"]+'"'
    df1 = top_movies.query(qry).sort_values('UserID', ascending=False)
    return df1.head(10).NAME.to_list()
    return prediction.tolist()

/opt/conda/lib/python3.7/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [13]:
modl_ob = register_model(None, score, name="Trending_Topic", 
                    description="Trending topics",
                    flavour=MLModelFlavours.sklearn)

In [17]:
print(modl_ob)

None
